In [ ]:
%pylab inline
#from pyspecdata import *
from serial.tools.list_ports import comports
import serial
import logging,os
#init_logging() # from pyspecdata --> goes to ~/pyspecdata.log

# the following just sets up a log file -- ignore
# (copy of pyspecdata init_logging)
def strm(*args):
    return ' '.join([str(j) for j in args])
FORMAT = "--> %(filename)s(%(lineno)s):%(name)s %(funcName)20s %(asctime)20s\n%(levelname)s: %(message)s"
log_filename = os.path.join(os.path.expanduser('~'),'pyspecdata.log')
if os.path.exists(log_filename):
    # manually remove, and then use append -- otherwise, it won't write to
    # file immediately
    os.remove(log_filename)
logging.basicConfig(format=FORMAT,
        filename=log_filename,
        filemode='a',
        )
logger = logging.getLogger('GDS_scope')
logger.setLevel(logging.DEBUG)

Here, I search through the comports in order to identify the instrument that I'm interested in.  This (or something like this) should work on either Windows or Mac/Linux.
I can use this to initialize a class.
Note that later, we might want to replace this with a class that relies on pyVISA rather than directly calling the serial connection.

In [ ]:
super?

In [ ]:
class SerialInstrument (object):
    """Class to describe an instrument connected using pyserial.
    Provides initialization (:func:`__init__`) to start the connection,
    as well as :func:`write` :func:`read` and :func:`respond` functions.
    Can be used inside a with block.
    """
    def __init__(self,textidn, **kwargs):
        """Initialize a serial connection based on the identifier string
        `textidn`, and assign it to the `connection` attribute

        Parameters
        ==========

        textidn : str
            
            A string used to identify the instrument.
            Specifically, the instrument responds to the ``*idn?`` command
            with a string that includes ``textidn``.
        """
        self.connection = serial.Serial(self.id_instrument(textidn), **kwargs)
        logger.debug('opened serial connection, and set to connection attribute')
        return
    def __enter__(self):
        return self
    def __exit__(self, exception_type, exception_value, traceback):
        self.connection.close()
        return
    def write(self,*args):
        """Send info to the instrument.  Take a comma-separated list of
        arguments, which are converted to strings and separated by a space.
        (Similar to a print command, but directed at the instrument)"""
        text = ' '.join([str(j) for j in args])
        logger.debug(strm("when trying to write, port looks like this:",self.connection))
        self.connection.write(text+'\n')
        return
    def read(self, *args, **kwargs):
        return self.connection.read(*args, **kwargs)
    def flush(self):
        """Flush the input (say we didn't read all of it, *etc.*)
        
        Note that there are routines called "flush" in serial, but these
        seem to not be useful.
        """
        old_timeout = self.connection.timeout
        self.connection.timeout = 1
        result = 'blah'
        while len(result)>0:
            result = self.connection.read(2000)
        self.connection.timeout = old_timeout
        return
    def respond(self,*args, **kwargs):
        """Same as write, but also returns the result
        
        Parameters
        ----------
        message_len : int

            If present, read a message of a specified number of bytes.
            if not present, return a text line (readline).
        """
        message_len = None
        if 'message_len' in kwargs:
            message_len = kwargs.pop('message_len')
        self.write(*args)
        old_timeout = self.connection.timeout
        self.connection.timeout = None
        if message_len is None:
            retval = self.connection.readline()
        else:
            retval = self.connection.read(message_len)
        self.connection.timeout = old_timeout
        return retval
    def id_instrument(self,textidn):
        """A helper function for :func:`init` Identify the instrument that returns an ID string containing ``textidn``
        """
        for j in comports():
            port_id = j[0] # based on the previous, this is the port number
            with serial.Serial(port_id) as s:
                s.write('*idn?\n')
                result = s.readline()
                if textidn in result:
                    return port_id

Note that I can now use ``SerialInstrument`` in place of ``serial.Serial``, except that ``SerialInstrument`` accepts a string identifying the instrument, and adds and changes various functions inside the class to make them more convenient.

In [ ]:
with SerialInstrument('GDS-3254') as s:
    logger.debug("running identify using the SerialInstrument class")
    logger.debug(strm("SerialInstrument instance looks like this:",s))
    print s.respond('*idn?')
    logger.debug("done running identify")

Next, we can 

In [ ]:
class GDS_scope (SerialInstrument):
    def __init__(self,model='3254'):
        super(self.__class__,self).__init__('GDS-'+model)
        logger.debug(strm("identify from within GDS",super(self.__class__,self).respond('*idn?')))
        logger.debug("I should have just opened the serial connection")
        return
    def waveform(self,instname='GDS-3254'):
        """Retrieve waveform and associated parameters form the scope.

        Comprises the following steps:

        * opens the port to the scope
        * acquires what is saved in memory as string
        * Divides this string at hashtag which separates settings from waveform

        Parameters
        ==========

        instname : str

            The instrument name.  Specifically, a string that's returned as
            part of the response to the ``*idn?`` command.

        Returns
        =======

        x_axis : ndarray

            The *x*-axis (time-base) of the data.

        data : ndarray

            A 1-d array containing the scope data.

        param : dict

            A dictionary of the parameters returned by the scope.
        """
        
        self.write(':ACQ1:MEM?')
        def upto_hashtag():
            this_char = self.read(1)
            this_line = ''
            while this_char != '#':          
                this_line += this_char
                this_char = self.read(1)
            return this_line

        #Further divides settings
        preamble = upto_hashtag().split(';')
        
        #Retrieves 'memory' of 25000 from settings
        #Waveform data is 50,000 bytes of binary data (2*mem)
        mem = int(preamble[0].split(',')[1])
        
        #Generates list of parameters in the preamble
        param = dict([tuple(x.split(',')) for x in preamble if len(x.split(',')) == 2])
        
        #Reads waveform data of 50,000 bytes
        self.read(6)# length of 550000
        data = self.read(50001)
        assert data[-1] == '\n', "data is not followed by newline!"
        data = data[:-1]

        # convert the binary string
        data_array = fromstring(data,dtype='i2')
        data_array =  double(data_array)/double(2**(2*8-1))

        # I could do the following
        #x_axis = r_[0:len(data_array)] * float(param['Sampling Period'])
        # but since I'm "using up" the sampling period, do this:
        x_axis = r_[0:len(data_array)] * float(param.pop('Sampling Period'))
        # r_[... is used by numpy to construct arrays on the fly

        # Similarly, use V/div scale to scale the y values of the data
        data_array *= float(param.pop('Vertical Scale'))/0.2 # we saw
        #              empirically that 0.2 corresponds to about 1 division

        return x_axis,data_array,param

The previous cell only ***defines*** the function ``retrieve_waveform``.  Now I have to actually call (run) it!:

In [ ]:
with GDS_scope() as g:
    g.flush()
    x_axis,data,param = g.waveform()

Next, show what the ``param`` dictionary looks like (note that the items that we've popped have been removed)

In [ ]:
param

Plots waveform data → I manually scale by μs

In [ ]:
title(param['Source'])
plot(x_axis/1e-6,data)
xlabel(r'$t$ / $\mu s$')

I'm not worrying about automatically interpreting the units here, because that will all be handled when we make our special data object.

I do want to check that I'm interpreting the y scale correctly, so let's look at the peak to peak voltage here, and compare to what we measure with cursors on the screen:

In [ ]:
data.max()-data.min()

the following is just so I can also run this as a script

In [ ]:
show()